In [ ]:
%aiida
from pybis import Openbis
from aiida_openbis.utils import bisutils,structures
import yaml
import pathlib
import collections
from copy import deepcopy

In [ ]:
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")

In [ ]:
obj = session.get_object("/MATERIALS/SAMPLES/TTC41")

In [ ]:
obj.props['name']

In [ ]:
property_type = session.get_property_type("smiles")

## Load object_types and property_types from yml file

In [ ]:
the_objects = './object_types.yml'
with open(the_objects,mode='r') as fhandle:
    items = yaml.safe_load(fhandle)

## Property types

In [ ]:
# delete existing
for key, value in items['property_types'].items():
    try:
        property_type = session.get_property_type(code=key)
        print(f"Property type '{key}' exists, deleting")
        property_type.delete("no reason.")
    except ValueError:
        print(f"Property type '{key}' does not exist")

In [ ]:
for key, value in items['property_types'].items():
    try:
        print(key)
        session.new_property_type(code=key, **value).save()
    except ValueError as err:
        print(err)

## Object types

In [ ]:
# delete objects
for key, value in items['object_types'].items():
    try:
        object_type = session.get_object_type(key)
        print(f"Property type '{key}' exists, deleting")
        object_type.delete("no reason.")
    except ValueError:
        print(f"Object type '{key}' does not exist")

In [ ]:
for key, value in deepcopy(items['object_types']).items():
    print(key)
    try:
        sections = value.pop('sections')
        object_type = session.new_object_type(code=key,
                                              autoGeneratedCode=True,
                                              subcodeUnique=False,
                                              listable=True,
                                              showContainer=False,
                                              showParents=True,
                                              showParentMetadata=False,
                                              **value)
        object_type.save()
        for section, properties in sections.items():
            print(section, properties)
            for property_type in properties:
                object_type.assign_property(session.get_property_type(code=property_type))
    except ValueError as exc:
        print(f"The property {key} seem to already be present")
        print(str(exc))


In [ ]:
for the_object in objects:
    object_type = session.new_object_type(
        code                = the_object, #'MOLECULE'
        generatedCodePrefix = objects[the_object]['code'], #'MOL'
        autoGeneratedCode   = True,
        subcodeUnique       = False,
        listable            = True,
        showContainer       = False,
        showParents         = True,
        showParentMetadata  = False,
        validationPlugin    = ''          # see plugins below
    )
    object_type.save()
    sections = list(objects[the_object]['sections'].keys())
    for section in sections[::-1]: # objects[the_object]['sections']:
        for prop in objects[the_object]['sections'][section]:
            metaData={}
            dataType = objects[the_object]['sections'][section][prop][2]
            print(dataType)
            if dataType == 'MULTILINE_VARCHAR':
                print('Word processor')
                metaData = {'custom_widget' : 'Word Processor'}
            elif dataType == 'XML':
                metaData    = {'custom_widget': 'Spreadsheet'}                
            loc_prop = session.new_property_type(
                code = the_object+'.'+prop,
                label = objects[the_object]['sections'][section][prop][0],
                description = objects[the_object]['sections'][section][prop][1],
                dataType = dataType,
                metaData    = metaData,
            )
            loc_prop.save()
            print(loc_prop)
            print(metaData)
            object_type.assign_property(
                prop                 = the_object+'.'+prop,           # mandatory
                section              = section,
                ordinal              = objects[the_object]['sections'][section][prop][4],
                mandatory            = objects[the_object]['sections'][section][prop][3],
                initialValueForExistingEntities = 'initial value',
                showInEditView       = True,
                showRawValueInForms  = True, 
            )

In [ ]:
for section in objects[the_object]['sections'][::-1]:
    print(section)

In [ ]:
for k in keys[::-1]:
    print(k)

In [ ]:
objects['polecule']

In [ ]:
import json
print(json.dumps(objects, indent=2))
print

# dump ordereddict to yaml
output = yaml.dump(objects, Dumper=Dumper, default_flow_style=False)
print(output)

In [ ]:
# directly write to a file object to save memory.
with open('result.yml', 'w') as f:
    yaml.dump(yml_dict, f, default_flow_style=False)

In [ ]:
sample_type = session.new_sample_type(
    code                = 'molecule',  # mandatory
    generatedCodePrefix = 'mol',                   # mandatory
    autoGeneratedCode   = True,
    subcodeUnique       = False,
    listable            = True,
    showContainer       = False,
    showParents         = True,
    showParentMetadata  = False,
)
sample_type.save()

In [ ]:
sample_type = session.new_sample_type(
    code                = 'solecule',  # mandatory
    generatedCodePrefix = 'sol',                   # mandatory
    autoGeneratedCode   = True,
    subcodeUnique       = False,
    listable            = True,
    showContainer       = False,
    showParents         = True,
    showParentMetadata  = False,
)
sample_type.save()

In [ ]:
sample_type.delete("aosdfasdf")

In [ ]:
property_type = session.new_property_type(
    code='smiles',
    label='smiles',
    description='smiles description',
    dataType='VARCHAR',
)
property_type.save()

In [ ]:
property_type.delete("asdfasdf")

In [ ]:
sample_type.assign_property(property_type)

In [ ]:
obj = session.get_object("/MATERIALS/SAMPLES/MOL7")

In [ ]:
print(obj)

In [ ]:
obj.props["info"]

In [ ]:
property_type = session.get_property_type("smiles")

In [ ]:
sample_type.assign_property(property_type)

In [ ]:
sample_types = session.get_sample_types()  # get a list of sample types
my_own = sample_types[13]
chemsketch = sample_types[14]
molecule = sample_types[15]
my_own.delete("asdf")
chemsketch.delete("asdf")
molecule.delete("asdfa")